In [9]:
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
from datetime import timedelta, date

In [11]:
credentials = service_account.Credentials.from_service_account_file('watchdog_private_key.json', scopes=["https://www.googleapis.com/auth/cloud-platform"])

view_name = 'ga_jp_product_daily_kpi'
primary_diamention = 'Product_Name'
diamention_item = 'LR トートバッグ'
csv_file = 'Japan Product Name.csv'

project_id = 'watchdog-340107'
dataset_id = 'levis_jp_watchdog'
view_id = f'{project_id}.{dataset_id}.{view_name}'

date_col = 'Date'
start_date = date(2022,6,20) 
end_date = date(2022,7,20)
date_range = pd.date_range(start_date,end_date-timedelta(days=1),freq='d')


query_string_dim = f"""
        SELECT
        *
        FROM `{project_id}.{dataset_id}.{view_name}` 
        WHERE ({date_col} BETWEEN '{start_date}' AND '{end_date}') AND {primary_diamention} = '{diamention_item}'
        ORDER BY {date_col}
        """

query_string = f"""
        SELECT
        *
        FROM `{project_id}.{dataset_id}.{view_name}` 
        WHERE {date_col} BETWEEN '{start_date}' AND '{end_date}' 
        ORDER BY {date_col}
        """

In [12]:
client = bigquery.Client(credentials=credentials, project=project_id)

In [6]:
# view_df.Date.values[0]

In [7]:
# csv_df.Date.values[0]

In [8]:
# view_df.dtypes

NameError: name 'view_df' is not defined

In [13]:
print("Start Date:",start_date," End Date:", end_date)

view_df = client.query(query_string_dim).result().to_dataframe()
view_df['Date'] = pd.to_datetime(view_df.Date, format='%Y-%m-%d')
print("View Dataframe Head: \n",view_df.head())
print("View Dataframe Size: (Rows, Columns)",view_df.shape)

csv_df = pd.read_csv('Japan Product Name.csv')
csv_df['Date'] = pd.to_datetime(csv_df.Date, format='%Y-%m-%d')
print("CSV Dataframe Head: \n",csv_df.head())
print("CSV Dataframe Size: (Rows, Columns)",csv_df.shape)

Start Date: 2022-06-20  End Date: 2022-07-20
View Dataframe Head: 
         Date Product_Name  PDP_Views  Orders  add_to_cart  Revenue  Quantity
0 2022-06-20    LR トートバッグ       29.0     0.0          2.0      0.0       0.0
1 2022-06-21    LR トートバッグ       13.0     0.0          0.0      0.0       0.0
2 2022-06-22    LR トートバッグ       18.0     0.0          0.0      0.0       0.0
3 2022-06-23    LR トートバッグ       30.0     0.0          0.0      0.0       0.0
4 2022-06-24    LR トートバッグ       44.0     0.0          0.0      0.0       0.0
View Dataframe Size: (Rows, Columns) (31, 7)
CSV Dataframe Head: 
         Date                                       Product_Name  Revenue  \
0 2021-07-05  1880モデル TRIPLE PLEAT BLOUSE /CONE DENIM/WHITE ...      0.0   
1 2021-07-05                  1880モデル TRIPLE PLEAT BLOUSE RIGID      0.0   
2 2021-07-05                         1890モデル XX501® JEANS RIGID      0.0   
3 2021-07-05                 1890モデル/501XX/WHITE OAK/リジッド/8.8oz      0.0   
4 2021-07-05           

In [103]:
daterange = pd.date_range(start_date,end_date-timedelta(days=1),freq='d')
print(daterange)

DatetimeIndex(['2022-06-20', '2022-06-21', '2022-06-22', '2022-06-23',
               '2022-06-24', '2022-06-25', '2022-06-26', '2022-06-27',
               '2022-06-28', '2022-06-29', '2022-06-30', '2022-07-01',
               '2022-07-02', '2022-07-03', '2022-07-04', '2022-07-05',
               '2022-07-06', '2022-07-07', '2022-07-08', '2022-07-09',
               '2022-07-10', '2022-07-11', '2022-07-12', '2022-07-13',
               '2022-07-14', '2022-07-15', '2022-07-16', '2022-07-17',
               '2022-07-18', '2022-07-19'],
              dtype='datetime64[ns]', freq='D')


In [14]:
mask = ((csv_df[date_col] > str(start_date)) & (csv_df[date_col] <= str(end_date)) & (csv_df[primary_diamention] == diamention_item))
masked_df = csv_df.loc[mask]
print(csv_df.shape,masked_df.shape)

(677662, 7) (13, 7)


In [15]:
masked_df.sort_values(by=date_col,ascending=False)

,Date,Product_Name,Revenue,Orders,Units,Product_Views,Cart_Adds
677549,2022-07-03,LR トートバッグ,9275.5074,1,2,53,2
675323,2022-07-02,LR トートバッグ,4623.4684,1,1,58,5
672421,2022-07-01,LR トートバッグ,0.0000,0,0,33,0
671748,2022-06-30,LR トートバッグ,5676.8540,1,1,41,4
670073,2022-06-29,LR トートバッグ,17093.9142,3,3,52,17
666488,2022-06-28,LR トートバッグ,0.0000,0,0,24,0
664432,2022-06-27,LR トートバッグ,0.0000,0,0,25,0
662318,2022-06-26,LR トートバッグ,0.0000,0,0,63,1
660253,2022-06-25,LR トートバッグ,0.0000,0,0,31,0
658126,2022-06-24,LR トートバッグ,0.0000,0,0,44,0


In [16]:
view_df.count()

Date            31
Product_Name    31
PDP_Views       31
Orders          31
add_to_cart     31
Revenue         31
Quantity        31
dtype: int64

In [ ]:
print(query_string_dim)

In [99]:
bq_cols = view_df.dtypes
# bq_cols = view_df.columns
bq_cols.size

7

In [98]:
for i in bq_cols.iteritems():
    print(i[1])

datetime64[ns]
object
float64
float64
float64
float64
float64


In [94]:
view_df.dtypes

Date            datetime64[ns]
Product_Name            object
PDP_Views              float64
Orders                 float64
add_to_cart            float64
Revenue                float64
Quantity               float64
dtype: object

In [89]:
view_df.columns

Index(['Date', 'Product_Name', 'PDP_Views', 'Orders', 'add_to_cart', 'Revenue',
       'Quantity'],
      dtype='object')

In [91]:
for n,t in zip(view_df.columns, view_df.dtypes):
    print(n,t)

Date datetime64[ns]
Product_Name object
PDP_Views float64
Orders float64
add_to_cart float64
Revenue float64
Quantity float64


Date
Product_Name
PDP_Views
Orders
add_to_cart
Revenue
Quantity


In [24]:
bq_cols.size

7

In [101]:
view_df[[date_col, 'PDP_Views']]

,Date,PDP_Views
0,2022-06-20,29.0
1,2022-06-21,13.0
2,2022-06-22,18.0
3,2022-06-23,30.0
4,2022-06-24,44.0
5,2022-06-25,31.0
6,2022-06-26,63.0
7,2022-06-27,25.0
8,2022-06-28,24.0
9,2022-06-29,52.0


In [102]:
masked_df[date_col, 'Product_Views']

KeyError: ('Date', 'Product_Views')

In [ ]:
df = view_df[[date_col, 'PDP_Views']]
d = masked_df[[date_col, 'Product_Views']]

df = df[df[date_col].notnull()]
d = d[d[date_col].notnull()]
# .set_index(date_col)

# df.join(d, on = date_col, how='left')
final_df = df.merge(d, on = date_col, how='left').sort_values(by=date_col,ascending=False)


In [ ]:
final_df.fillna(0)

In [ ]:
final_df["% Error"]=((final_df["PDP_Views"] - final_df["Product_Views"])/final_df["PDP_Views"])*100

In [ ]:
d.dtypes


In [ ]:
df.dtypes

In [ ]:
final_df